## Analyze recipe directions

* Tokenize directions for each recipe, create bow field
* Basic TFIDF over all recipes

In [1]:
import pymongo 
import boto3 #go get 'about_me' page 
from bs4 import BeautifulSoup as BS
import pickle

In [2]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
allrecipes_db = client.allrecipes
allrecipes_db.collection_names()

[u'community_link_pages',
 u'recipes',
 u'members',
 u'webpages',
 u'healthy_recipes',
 u'community_page']

In [3]:
members_coll = allrecipes_db.members

In [4]:
recipes_coll = allrecipes_db.recipes

In [5]:
# Get all the keys 
recipes_coll.find_one().keys()

[u'recipe_times',
 u'nutrition_elements',
 u'name',
 u'nutrition_serving_info',
 u'description',
 u'servings_config',
 u'recipe_ID',
 u'rating_list',
 u'servings',
 u'nutrition_info',
 u'_id',
 u'page',
 u'directions_list',
 u'ingredients_list']

In [6]:
allrecipes_db.command({'dbstats': 1})

{u'avgObjSize': 295119.7292980532,
 u'collections': 5,
 u'dataSize': 4305206611.0,
 u'db': u'allrecipes',
 u'indexSize': 397312.0,
 u'indexes': 6,
 u'numExtents': 0,
 u'objects': 14588,
 u'ok': 1.0,
 u'storageSize': 1491963904.0,
 u'views': 0}

In [7]:
list(recipes_coll.find({"recipe_ID": "56188094"}, ["recipe_ID", "name", "directions_list"]))

[{u'_id': ObjectId('58d9f43e1bc9b2453dcc2041'),
  u'directions_list': [u'Mix the flour and salt together in a large bowl or in the bowl of a stand mixer that is fitted with a dough hook. Gradually stir in water until the dough holds together enough to come loose from the sides of the bowl. You may not need the full amount of water. Shape into a flat ball, and allow to rest for at least 10 minutes.',
   u'Place the butter between two pieces of plastic wrap and pound with a rolling pin into a flat disc. Refrigerate until firm, about 20 minutes.',
   u"On a lightly floured surface, roll out the dough into a large rectangle about 1/2 inch thick. Place the disc of chilled butter in the center and fold the two ends over it so that it is completely encased in dough. Roll out the dough again, taking care not to let the butter break through the dough, to about 1/2 inch thickness. Fold into thirds. This is the first ''turn''.  Rotate the dough 90 degrees and roll out into a rectangle again. Fold

In [10]:
recipes_coll.find_one()['directions_list']

[u'Preheat oven to 350 degrees F (175 degrees C).',
 u'In a medium bowl, cream together the butter, white sugar and brown sugar until smooth. Stir in the peanut butter, vanilla and egg until well blended. Combine the flour, baking soda and salt; stir into the batter just until moistened. Mix in the oats and chocolate chips until evenly distributed. Drop by tablespoonfuls on to lightly greased cookie sheets.',
 u'Bake for 10 to 12 minutes in the preheated oven, until the edges start to brown. Cool on cookie sheets for about 5 minutes before transferring to wire racks to cool completely.',
 u'']

In [11]:
print "\n".join(recipes_coll.find_one()['directions_list'])

Preheat oven to 350 degrees F (175 degrees C).
In a medium bowl, cream together the butter, white sugar and brown sugar until smooth. Stir in the peanut butter, vanilla and egg until well blended. Combine the flour, baking soda and salt; stir into the batter just until moistened. Mix in the oats and chocolate chips until evenly distributed. Drop by tablespoonfuls on to lightly greased cookie sheets.
Bake for 10 to 12 minutes in the preheated oven, until the edges start to brown. Cool on cookie sheets for about 5 minutes before transferring to wire racks to cool completely.



In [12]:
recipe_name_list = list(recipes_coll.find({"name":
                                            {"$exists": True}},['name']))

In [13]:
recipe_name_list = [a['name'] for a in recipe_name_list]

In [14]:
recipe_name_list[:40]

[[u'Outrageous Chocolate Chip Cookies'],
 [u'Cranberry Almond Biscotti'],
 [u'Pecan Crescents'],
 [u'Balsamic Chicken  '],
 [u"The Rebbetzin Chef's Persian Walnut Cookies"],
 [u"Grandma's Gingersnap Cookies"],
 [u'Cowboy Caviar'],
 [u'Pumpkin Cookies with Penuche Frosting'],
 [u'Almond Squares II'],
 [u'The Best Rolled Sugar Cookies'],
 [u'Faux Bombay Potatoes'],
 [u"Mimi's Zucchini Pie"],
 [u'Cookie Pralines'],
 [u'Peanut Butter Sandwich Cookies'],
 [u'Cranberry Pistachio Biscotti'],
 [u'Orange Almond Biscotti I'],
 [u'Pollo (Chicken) Fricassee from Puerto Rico'],
 [u'Cowboy Oatmeal Cookies'],
 [u'Costa Rican Coffee Panna Cotta with Bittersweet Chocolate-Rum Sauce'],
 [u'Chocolate Toffee Cookies II'],
 [u'Mayonnaise Cookies'],
 [u'Chewy Maple Cookies'],
 [u'Oatmeal Craisin Cookies'],
 [u'Aebleskiver'],
 [u'Buckeye Balls'],
 [u'Snowballs II'],
 [u'Caramel Macchiato Cheesecake'],
 [u"Grammy Burnham's Molasses Cookies"],
 [u'Cool Whip Cookies'],
 [u'Saltine Toffee Cookies'],
 [u'Chewy Gr

In [15]:
directions_list = list(recipes_coll.find({"name": 
                                            {"$exists": True}},['directions_list']))

In [16]:
directions_list[0]

{u'_id': ObjectId('58d9ee6a1bc9b2453dcc1579'),
 u'directions_list': [u'Preheat oven to 350 degrees F (175 degrees C).',
  u'In a medium bowl, cream together the butter, white sugar and brown sugar until smooth. Stir in the peanut butter, vanilla and egg until well blended. Combine the flour, baking soda and salt; stir into the batter just until moistened. Mix in the oats and chocolate chips until evenly distributed. Drop by tablespoonfuls on to lightly greased cookie sheets.',
  u'Bake for 10 to 12 minutes in the preheated oven, until the edges start to brown. Cool on cookie sheets for about 5 minutes before transferring to wire racks to cool completely.',
  u'']}

In [20]:
name_cur = recipes_coll.find({"directions_list": {"$exists": True}},['name'])
directions_cur = recipes_coll.find({"directions_list": {"$exists": True}},['directions_list'])

In [31]:
# use this to observe ingredients lists 
print name_cur.next()['name'], "\n"
print "\n".join(directions_cur.next()['directions_list'])

[u'Faux Bombay Potatoes'] 

Place the turnips into a large pot and cover with salted water. Bring to a boil over high heat, then reduce heat to medium-low, cover, and simmer until tender, 15 to 20 minutes. Drain and allow to steam dry for a minute or two.
Heat the oil in a large skillet over medium-high heat. Fry the yellow mustard seeds, black mustard seeds, and turmeric in the oil until the mustard seeds begin to pop. Add the turnips to the skillet; cook and stir until the turnips are completely heated through, about 5 minutes. Season with salt to serve.



---
### First-pass predictions using TF-IDF 
* Create directions_bow for all recipes 
* tf-idf over all recipes 


In [32]:
from sklearn.feature_extraction.text import TfidfTransformer

In [39]:
directions_docs = [a['directions_list'] for a in directions_list]

In [40]:
len(directions_docs)

9023

In [41]:
directions_corpus = [' \n'.join(a) for a in directions_docs]

In [35]:
from sklearn.feature_extraction.text import CountVectorizer 

In [95]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]


In [96]:
count_vect = CountVectorizer(stop_words="english", tokenizer=LemmaTokenizer()) 

In [97]:
directions_counts = count_vect.fit_transform(directions_corpus)

In [98]:
directions_counts.shape

(9023, 6407)

In [99]:
directions_corpus[0]

u'Preheat oven to 350 degrees F (175 degrees C). \nIn a medium bowl, cream together the butter, white sugar and brown sugar until smooth. Stir in the peanut butter, vanilla and egg until well blended. Combine the flour, baking soda and salt; stir into the batter just until moistened. Mix in the oats and chocolate chips until evenly distributed. Drop by tablespoonfuls on to lightly greased cookie sheets. \nBake for 10 to 12 minutes in the preheated oven, until the edges start to brown. Cool on cookie sheets for about 5 minutes before transferring to wire racks to cool completely. \n'

In [100]:
tfidf_transformer = TfidfTransformer()
directions_tfidf = tfidf_transformer.fit_transform(directions_counts)


In [102]:
count_vect.get_feature_names()

[u'!',
 u'#',
 u'%',
 u'&',
 u"'",
 u"''",
 u"''brain",
 u"''turn",
 u"''turns",
 u"'bake",
 u"'baking",
 u"'beard",
 u"'bed",
 u"'boat",
 u"'body",
 u"'braid",
 u"'burger-sized",
 u"'buttons",
 u"'cracks",
 u"'cream",
 u"'croutons",
 u"'crown",
 u"'cupcakes",
 u"'d",
 u"'eyes",
 u"'face",
 u"'feet",
 u"'fur",
 u"'garland",
 u"'good",
 u"'hat",
 u"'head",
 u"'high",
 u"'jellyroll-style",
 u"'ll",
 u"'log",
 u"'logs",
 u"'m",
 u"'muffin",
 u"'noodles",
 u"'party",
 u"'peanut",
 u"'petals",
 u"'pieces",
 u"'pizza",
 u"'pom-pom",
 u"'porridge",
 u"'re",
 u"'rice",
 u"'roux",
 u"'s",
 u"'shell",
 u"'shredded",
 u"'sloppy",
 u"'smoke",
 u"'sour",
 u"'springy",
 u"'stars",
 u"'steaks",
 u"'tail",
 u"'technique",
 u"'tree",
 u"'ve",
 u"'well",
 u"'x",
 u'(',
 u')',
 u'*',
 u'**',
 u'***',
 u'**dressing**',
 u'**salad**',
 u'**see',
 u'*add',
 u'*bake',
 u'*bring',
 u'*cook',
 u'*crumple',
 u'*drain',
 u'*fills',
 u'*glass',
 u'*halve',
 u'*heat',
 u'*i',
 u'*keep',
 u'*let',
 u'*meanwhile',
 

In [135]:
directions_tfidf.data.shape

(557333,)

In [103]:
from sklearn.decomposition import TruncatedSVD, NMF

In [125]:
nmf = NMF(n_components=3)

In [126]:
W_sklearn = nmf.fit_transform(directions_tfidf)
H_sklearn = nmf.components_

In [127]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from numpy.random import rand, RandomState
from numpy import array, matrix, linalg

In [128]:
feature_names= count_vect.get_feature_names()

In [129]:
def reconst_mse(target, left, right):
    return (array(target - left.dot(right))**2).mean()

def describe_nmf_results(document_term_mat, W, H, n_top_words = 15):
    print("Reconstruction error: %f") %(reconst_mse(document_term_mat, W, H))
    for topic_num, topic in enumerate(H):
        print("Topic %d:" % topic_num)
        print(" ".join([count_vect.get_feature_names()[i] \
                for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [130]:
describe_nmf_results(directions_tfidf, W_sklearn, H_sklearn)

Reconstruction error: 0.000119
Topic 0:
. heat cook , minute stir chicken skillet ; add boil bring simmer 5 medium
Topic 1:
. degree , oven baking ) ( f pan c bake flour sugar egg preheat
Topic 2:
, . pepper bowl garlic juice salt hour powder onion oil sauce vinegar mix combine


In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_transformer = TfidfVectorizer(encoding='utf-8')
directions_tfidf = tf_transformer.fit_transform(directions_corpus)
directions_tfidf.shape

(9023, 6018)

In [77]:
tf_transformer.get_feature_names()[300:450]

[u'7x11',
 u'7x3',
 u'7x4',
 u'80',
 u'82',
 u'85',
 u'88',
 u'89',
 u'8g',
 u'8inch',
 u'8min',
 u'8th',
 u'8x11',
 u'8x12',
 u'8x13',
 u'8x3',
 u'8x4',
 u'8x4x2',
 u'8x8',
 u'8x8in',
 u'8x8x2',
 u'8x8x3',
 u'90',
 u'90mg',
 u'95',
 u'96',
 u'97',
 u'98',
 u'99',
 u'9g',
 u'9inch',
 u'9x11',
 u'9x12',
 u'9x13',
 u'9x14',
 u'9x18',
 u'9x2x1',
 u'9x3',
 u'9x4',
 u'9x5',
 u'9x5x3',
 u'9x9',
 u'able',
 u'about',
 u'about1',
 u'above',
 u'abracadabra',
 u'absorb',
 u'absorbed',
 u'absorbs',
 u'abut',
 u'accept',
 u'accidentally',
 u'accomadate',
 u'accommodate',
 u'accompanied',
 u'according',
 u'accordingly',
 u'accordion',
 u'accumulate',
 u'accumulated',
 u'accumulates',
 u'acdc',
 u'achieve',
 u'achieved',
 u'achieves',
 u'acid',
 u'acini',
 u'acni',
 u'acorn',
 u'across',
 u'act',
 u'action',
 u'activate',
 u'activates',
 u'active',
 u'actual',
 u'actually',
 u'adaptable',
 u'add',
 u'add1',
 u'added',
 u'adding',
 u'addition',
 u'additional',
 u'additions',
 u'additonal',
 u'adds',
 

In [79]:
from sklearn.naive_bayes import MultinomialNB

In [80]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [179]:
ingredient_wordcounts = count_vect.fit_transform(ingredients_corpus)

In [180]:
ingredient_wordcounts.shape

(9023, 3647)

In [181]:
count_vect.vocabulary_.get('paprika')

2353

In [182]:
count_vect.vocabulary_.get?

In [183]:
dict(k = 5).get('k')

5

In [184]:
count_vect.vocabulary_['cilantro']

838

In [185]:
from sklearn.feature_extraction.text import TfidfTransformer

In [186]:
tf_transformer = TfidfTransformer().fit(ingredient_wordcounts)

In [187]:
ingredient_tfidf = tf_transformer.transform(ingredient_wordcounts)

In [188]:
ingredient_tfidf.shape

(9023, 3647)

In [189]:
ingredient_tfidf[0,:]

<1x3647 sparse matrix of type '<type 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [190]:
sorted(count_vect.vocabulary_, key=count_vect.vocabulary_.get, reverse=True)

[u'\xbetsp',
 u'\xbdtsp',
 u'\xbctsp',
 u'\xbccup',
 u'\xbaf',
 u'zucchinis',
 u'zucchini',
 u'zote',
 u'ziti',
 u'zip',
 u'zinfandel',
 u'zesty',
 u'zested',
 u'zest',
 u'zapallo',
 u'yummy',
 u'yukon',
 u'yucca',
 u'yuca',
 u'your',
 u'young',
 u'you',
 u'york',
 u'yolks',
 u'yolk',
 u'yogurt',
 u'yoghurt',
 u'yet',
 u'yes',
 u'yellowfin',
 u'yellow',
 u'yeast',
 u'yards',
 u'yams',
 u'yam',
 u'yacon',
 u'xylitol',
 u'xanthan',
 u'x2',
 u'wraps',
 u'wrappers',
 u'wrapped',
 u'wrap',
 u'would',
 u'worth',
 u'worms',
 u'world',
 u'works',
 u'work',
 u'wordpress',
 u'worchestershire',
 u'worchester',
 u'worcestershire',
 u'worcester',
 u'wooden',
 u'wood',
 u'wontons',
 u'wonton',
 u'won',
 u'wiz',
 u'without',
 u'within',
 u'with',
 u'wish',
 u'wise',
 u'wire',
 u'wip',
 u'winter',
 u'wingtips',
 u'wings',
 u'wingettes',
 u'wing',
 u'wine',
 u'wilton',
 u'williamsburg',
 u'will',
 u'wildflower',
 u'wildfire',
 u'wild',
 u'wieners',
 u'width',
 u'wider',
 u'wide',
 u'wholemeal',
 u'whol

In [191]:
import nltk
from nltk import word_tokenize, pos_tag

In [238]:
ingredients_docs[:2]

[[u'1/2 cup butter',
  u'1/2 cup white sugar',
  u'1/3 cup packed brown sugar',
  u'1/2 cup peanut butter',
  u'1/2 teaspoon vanilla extract',
  u'1 egg',
  u'1 cup all-purpose flour',
  u'1 teaspoon baking soda',
  u'1/4 teaspoon salt',
  u'1/2 cup rolled oats',
  u'1 cup semisweet chocolate chips'],
 [u'2 1/4 cups all-purpose flour',
  u'1 cup white sugar',
  u'1 teaspoon baking powder',
  u'1/2 teaspoon baking soda',
  u'2 egg whites',
  u'2 eggs',
  u'1 tablespoon vanilla extract',
  u'3/4 cup sliced almonds',
  u'1 cup sweetened-dried cranberries']]

In [262]:
# Make ingredients list into a list of lists 
ingredientsinlist = [[[a] for a in b] for b in ingredients_docs]

In [263]:
ingredientsinlist[0]

[[u'1/2 cup butter'],
 [u'1/2 cup white sugar'],
 [u'1/3 cup packed brown sugar'],
 [u'1/2 cup peanut butter'],
 [u'1/2 teaspoon vanilla extract'],
 [u'1 egg'],
 [u'1 cup all-purpose flour'],
 [u'1 teaspoon baking soda'],
 [u'1/4 teaspoon salt'],
 [u'1/2 cup rolled oats'],
 [u'1 cup semisweet chocolate chips']]

In [273]:
# Tokenize this list of lists 
tokenized_inglistinlist =[[nltk.pos_tag(word_tokenize(a)) for a in b] for b in ingredients_docs]


In [281]:

ingredients_docs[3]

[u'1/3 cup balsamic vinegar',
 u'1/2 cup chicken broth',
 u'2 tablespoons white sugar',
 u'1 clove garlic, minced',
 u'1 teaspoon dried Italian herb seasoning',
 u'4 skinless, boneless chicken breast halves',
 u'1 tablespoon olive oil']

In [298]:
tokenized_inglistinlist[0][0][0]

(u'1/2', 'CD')

In [300]:
import collections

In [307]:
from collections import defaultdict

In [309]:
collections.Counter?

In [330]:
unit_counter  = []
for j, a in enumerate(tokenized_inglistinlist): 
    for i,b in enumerate(a): 
        try:
            unit_counter.append(b[1])
        except: 
            print j,i,b

146 4 [(u'skewers', 'NNS')]
310 8 [(u'skewers', 'NNS')]
339 3 [(u'toothpicks', 'NNS')]
367 4 [(u'toothpicks', 'NNS')]
411 7 [(u'salt', 'NN')]
529 8 [(u'Salt', 'NN')]
545 0 [(u'ice', 'NN')]
727 0 [(u'Vinaigrette', 'NN')]
727 12 [(u'Vegetables', 'NNS')]
756 6 [(u'cornstarch', 'NN')]
765 1 [(u'salt', 'NN')]
768 7 [(u'MARINADE', 'NN')]
815 2 [(u'egg', 'NN')]
815 3 [(u'onion', 'NN')]
815 4 [(u'garlic', 'NN')]
848 3 [(u'Ice', 'NN')]
1021 10 [(u'skewers', 'NNS')]
1058 0 [(u'PUDDING', 'NN')]
1066 6 [(u'pepper', 'NN')]
1216 9 [(u'skewers', 'NNS')]
1324 6 [(u'Paprika', 'FW')]
1549 11 [(u'Guacamole', 'NN')]
1562 10 [(u'TOPPING', 'NN')]
1568 1 [(u'Eggs', 'NNS')]
1568 3 [(u'Water', 'NN')]
1576 7 [(u'salt', 'NN')]
1787 3 [(u'salt', 'NN')]
1810 4 [(u'ice', 'NN')]
1825 2 [(u'pepper', 'NN')]
2290 7 [(u'Burgers', 'NNS')]
2435 0 [(u'MUFFINS', 'NN')]
2435 10 [(u'TOPPING', 'NN')]
2616 9 [(u'skewers', 'NNS')]
2668 6 [(u'skewers', 'NNS')]
2716 0 [(u'Marinade', 'NN')]
2774 10 [(u'Salsa', 'NN')]
2795 3 [(u'sal

In [326]:
ing_collcount = collections.Counter(unit_counter)

In [329]:
ing_collcount.most_common(30)

[((u'cup', 'NN'), 14478),
 ((u'teaspoon', 'NN'), 11382),
 ((u'(', '('), 6888),
 ((u'tablespoons', 'NNS'), 6054),
 ((u'cups', 'NNS'), 4211),
 ((u'tablespoon', 'NN'), 4207),
 ((u'cup', 'JJ'), 3415),
 ((u'1/2', 'CD'), 3290),
 ((u'teaspoons', 'NNS'), 2131),
 ((u'pound', 'NN'), 1562),
 ((u'eggs', 'NNS'), 1389),
 ((u'and', 'CC'), 1230),
 ((u'cloves', 'NNS'), 1156),
 ((u'large', 'JJ'), 1148),
 ((u'egg', 'NN'), 982),
 ((u'ounces', 'NNS'), 836),
 ((u'onion', 'NN'), 666),
 ((u'pounds', 'NNS'), 612),
 ((u'1/4', 'CD'), 556),
 ((u'pinch', 'NN'), 517),
 ((u'small', 'JJ'), 514),
 ((u'slices', 'NNS'), 485),
 ((u'green', 'JJ'), 481),
 ((u'tsp', 'NN'), 445),
 ((u'cup', 'NNS'), 418),
 ((u'clove', 'NN'), 395),
 ((u'medium', 'NN'), 367),
 ((u'red', 'JJ'), 347),
 ((u'pinch', 'JJ'), 347),
 ((u'to', 'TO'), 337)]

In [332]:
recipe_name_list[367]

[u'Stuffed Jalapeno Firecrackers']

In [319]:
tokenized_inglistinlist[146][4]

[(u'skewers', 'NNS')]

In [312]:
len(unit_counter)

0

In [289]:
quant = [b[0] for b in tokenized_inglistinlist]

IndexError: list index out of range

In [288]:
quant[0:20]


[u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1',
 u'1']

In [ ]:
pos_tok_inglistlist = [tokenized_inglistinlist

In [208]:
ingredients_corpus[0]

u'1/2 cup butter \n1/2 cup white sugar \n1/3 cup packed brown sugar \n1/2 cup peanut butter \n1/2 teaspoon vanilla extract \n1 egg \n1 cup all-purpose flour \n1 teaspoon baking soda \n1/4 teaspoon salt \n1/2 cup rolled oats \n1 cup semisweet chocolate chips'

In [212]:
il = [a.split() for a in ingredients_corpus]

In [213]:
il[0]

[u'1/2',
 u'cup',
 u'butter',
 u'1/2',
 u'cup',
 u'white',
 u'sugar',
 u'1/3',
 u'cup',
 u'packed',
 u'brown',
 u'sugar',
 u'1/2',
 u'cup',
 u'peanut',
 u'butter',
 u'1/2',
 u'teaspoon',
 u'vanilla',
 u'extract',
 u'1',
 u'egg',
 u'1',
 u'cup',
 u'all-purpose',
 u'flour',
 u'1',
 u'teaspoon',
 u'baking',
 u'soda',
 u'1/4',
 u'teaspoon',
 u'salt',
 u'1/2',
 u'cup',
 u'rolled',
 u'oats',
 u'1',
 u'cup',
 u'semisweet',
 u'chocolate',
 u'chips']

In [211]:
ingredients_list_tokenized[0]

[u'1/2',
 u'cup',
 u'butter',
 u'1/2',
 u'cup',
 u'white',
 u'sugar',
 u'1/3',
 u'cup',
 u'packed',
 u'brown',
 u'sugar',
 u'1/2',
 u'cup',
 u'peanut',
 u'butter',
 u'1/2',
 u'teaspoon',
 u'vanilla',
 u'extract',
 u'1',
 u'egg',
 u'1',
 u'cup',
 u'all-purpose',
 u'flour',
 u'1',
 u'teaspoon',
 u'baking',
 u'soda',
 u'1/4',
 u'teaspoon',
 u'salt',
 u'1/2',
 u'cup',
 u'rolled',
 u'oats',
 u'1',
 u'cup',
 u'semisweet',
 u'chocolate',
 u'chips']

In [192]:
ing_tok =[word_tokenize(a) for a in ingredients_corpus]

In [207]:
# Tokenize 

[u'1/2',
 u'cup',
 u'butter',
 u'1/2',
 u'cup',
 u'white',
 u'sugar',
 u'1/3',
 u'cup',
 u'packed',
 u'brown',
 u'sugar',
 u'1/2',
 u'cup',
 u'peanut',
 u'butter',
 u'1/2',
 u'teaspoon',
 u'vanilla',
 u'extract',
 u'1',
 u'egg',
 u'1',
 u'cup',
 u'all-purpose',
 u'flour',
 u'1',
 u'teaspoon',
 u'baking',
 u'soda',
 u'1/4',
 u'teaspoon',
 u'salt',
 u'1/2',
 u'cup',
 u'rolled',
 u'oats',
 u'1',
 u'cup',
 u'semisweet',
 u'chocolate',
 u'chips']

In [193]:
tagged = [nltk.pos_tag(a) for a in ing_tok]

In [194]:
tagged[0]

[(u'1/2', 'CD'),
 (u'cup', 'NN'),
 (u'butter', 'NN'),
 (u'1/2', 'CD'),
 (u'cup', 'NN'),
 (u'white', 'JJ'),
 (u'sugar', 'NN'),
 (u'1/3', 'CD'),
 (u'cup', 'NN'),
 (u'packed', 'VBD'),
 (u'brown', 'JJ'),
 (u'sugar', 'NN'),
 (u'1/2', 'CD'),
 (u'cup', 'NN'),
 (u'peanut', 'NN'),
 (u'butter', 'NN'),
 (u'1/2', 'CD'),
 (u'teaspoon', 'NN'),
 (u'vanilla', 'NN'),
 (u'extract', 'NN'),
 (u'1', 'CD'),
 (u'egg', 'NN'),
 (u'1', 'CD'),
 (u'cup', 'NN'),
 (u'all-purpose', 'JJ'),
 (u'flour', 'NN'),
 (u'1', 'CD'),
 (u'teaspoon', 'NN'),
 (u'baking', 'VBG'),
 (u'soda', 'NN'),
 (u'1/4', 'CD'),
 (u'teaspoon', 'NN'),
 (u'salt', 'VBD'),
 (u'1/2', 'CD'),
 (u'cup', 'NN'),
 (u'rolled', 'VBD'),
 (u'oats', 'NNS'),
 (u'1', 'CD'),
 (u'cup', 'JJ'),
 (u'semisweet', 'NN'),
 (u'chocolate', 'NN'),
 (u'chips', 'NNS')]

In [195]:
ingredients_docs[0]

[u'1/2 cup butter',
 u'1/2 cup white sugar',
 u'1/3 cup packed brown sugar',
 u'1/2 cup peanut butter',
 u'1/2 teaspoon vanilla extract',
 u'1 egg',
 u'1 cup all-purpose flour',
 u'1 teaspoon baking soda',
 u'1/4 teaspoon salt',
 u'1/2 cup rolled oats',
 u'1 cup semisweet chocolate chips']

In [203]:
quantities = []
for a in tagged: 
    c = [b[0] for b in a]
    quantities.append(c)

In [204]:
quantities[:100]

[[u'1/2',
  u'cup',
  u'butter',
  u'1/2',
  u'cup',
  u'white',
  u'sugar',
  u'1/3',
  u'cup',
  u'packed',
  u'brown',
  u'sugar',
  u'1/2',
  u'cup',
  u'peanut',
  u'butter',
  u'1/2',
  u'teaspoon',
  u'vanilla',
  u'extract',
  u'1',
  u'egg',
  u'1',
  u'cup',
  u'all-purpose',
  u'flour',
  u'1',
  u'teaspoon',
  u'baking',
  u'soda',
  u'1/4',
  u'teaspoon',
  u'salt',
  u'1/2',
  u'cup',
  u'rolled',
  u'oats',
  u'1',
  u'cup',
  u'semisweet',
  u'chocolate',
  u'chips'],
 [u'2',
  u'1/4',
  u'cups',
  u'all-purpose',
  u'flour',
  u'1',
  u'cup',
  u'white',
  u'sugar',
  u'1',
  u'teaspoon',
  u'baking',
  u'powder',
  u'1/2',
  u'teaspoon',
  u'baking',
  u'soda',
  u'2',
  u'egg',
  u'whites',
  u'2',
  u'eggs',
  u'1',
  u'tablespoon',
  u'vanilla',
  u'extract',
  u'3/4',
  u'cup',
  u'sliced',
  u'almonds',
  u'1',
  u'cup',
  u'sweetened-dried',
  u'cranberries'],
 [u'1',
  u'cup',
  u'butter',
  u'1/2',
  u'teaspoon',
  u'salt',
  u'2',
  u'cups',
  u'chopped',
  u

In [205]:
ingredients_corpus[0]

u'1/2 cup butter \n1/2 cup white sugar \n1/3 cup packed brown sugar \n1/2 cup peanut butter \n1/2 teaspoon vanilla extract \n1 egg \n1 cup all-purpose flour \n1 teaspoon baking soda \n1/4 teaspoon salt \n1/2 cup rolled oats \n1 cup semisweet chocolate chips'

---
NLP of `Description`

In [ ]:
ingredients_docs = [a['ingredients_list'] for a in ingredients_docs]